In [27]:
import pandas as pd
import os
import glob
import mygene 
# os.chdir(os.path.join('tissue_cocoputs','data'))

# print(os.listdir()
# )
# human_cds = pd.read_csv('o586358-Human_CDS_Bicod.tsv')

# human_cds

def subset_and_write(df,col_list,fn):
    df = df.set_index('transcript_id')
    df = df[col_list]
    df.to_csv(fn)

def create_tissue_directory(sample_data, tissues):
    sample_dir = {}
    for tissue in tissues:
        sample_dir[tissue] = sample_data.loc[sample_data['SMTSD'] == tissue]['SAMPID'].values.tolist()
    return(sample_dir)

def get_weights(geneSyms, gtex_avg):

    def append_results(transripts,refseqs,weights):
        sub_transcripts = [x for x in transripts if x in gtex_avg.index]
        weight = gtex_avg[sub_transcripts].sum()
        refseqs.append(gdict['query'])
        weights.append(weight)
        
    refseqs = []
    weights = []
    counter = 0
    for gdict in geneSyms:
        if 'ensembl' in gdict:
            if len(gdict['ensembl']) == 1:
                transripts = gdict['ensembl']['transcript']
                append_results(transripts,refseqs,weights)
            else:
                # if the results has a list for the 'transcript' dict key
                for ensmbl in gdict['ensembl']:
                    transripts = ensmbl['transcript']
                    append_results(transripts,refseqs,weights)

        counter +=1    

    return(refseqs,weights)

def generate_bicodon_table(refseqs,weights,file_id):

    bcds_sorted = bcds.loc[refseqs,:]
    bcds_weighted = (bcds_sorted.T * weights).sum(axis=1)
    bcds_weighted.index = bcds_weighted.index.str.upper()
#     bcds_weighted.to_frame().transpose().to_csv(
#         'bicodon_tables/GTEx_{}_{}_v7.tsv'.format(tissue,file_id), sep='\t', index=False)
    print('Generated Bicodon Table')
    return(bcds_weighted)


def generate_codon_table(refseqs,weights,file_id):

    cds_sorted = cds.loc[refseqs,:]
    cds_weighted = (cds_sorted.T * weights).sum(axis=1)

    # output codon freq table
    output = pd.read_csv('/grid/home/nbourgeois/codonOpt/references/codon_usage.getex.txt',sep='\t')
    output = output.drop(columns='Fraction')
    output.loc[:,'Number'] = cds_weighted.values.astype(int)
    output.loc[:,'Frequency'] = round(output['Number'] / sum(output['Number']) * 1000,1)
    output.to_csv('codon_tables/GTEx_{}_{}_v7.tsv'.format(tissue,file_id), sep='\t', index=False)
    print('Generated Codon Table')

In [2]:

ref_dir = os.path.join(os.getcwd(),'..','references')
os.chdir('/grid/home/nbourgeois/codonOpt/tissue_cocoputs/data')

# Original Data
gtex_data = pd.read_csv('TPM/GTEx_Liver_TPM_orig.tsv',sep='\t')
gtex_samp_data= pd.read_csv('ref/GTEx_v7_Annotations_SampleAttributesDS.txt',sep='\t')




In [3]:
gtex_samp_data = gtex_samp_data.loc[gtex_samp_data['SAMPID'].isin(gtex_data.columns)]

tissues = gtex_samp_data['SMTSD'].unique().tolist()
sample_dir = create_tissue_directory(gtex_samp_data,tissues)



In [4]:
bcds = pd.read_csv('o586358-Human_CDS_Bicod.tsv',sep='\t')
bcds = bcds.drop(columns = ['Gene ID', 'Accession', 'Division', 'Assembly', 'Taxid',
                        'Species', 'Organelle', '# Codon Pairs', 'GC%', 'GC1%', 'GC2%', 'GC3%','Unnamed: 4109'])
bcds = bcds.groupby('Protein ID').sum()

# convert protein info to ensmbl transcript info



In [73]:
gtex_data

,Unnamed: 0,transcript_id,gene_id,GTEX-11DXY-0526-SM-5EGGQ,GTEX-11DXZ-0126-SM-5EGGY,GTEX-11EQ9-0526-SM-5A5JZ,GTEX-11GSP-0626-SM-5986T,GTEX-11NUK-1226-SM-5P9GM,GTEX-11NV4-1326-SM-5HL6V,GTEX-11OF3-0726-SM-5BC4Z,...,GTEX-ZF29-2026-SM-4WWB7,GTEX-ZF2S-3026-SM-4WWCH,GTEX-ZPU1-0826-SM-57WG2,GTEX-ZTPG-1426-SM-51MT3,GTEX-ZVP2-0626-SM-51MSO,GTEX-ZVT3-1626-SM-5GU66,GTEX-ZVT4-0626-SM-5E45T,GTEX-ZYT6-0626-SM-5E45V,GTEX-ZYY3-0626-SM-5NQ6W,GTEX-ZZPU-0426-SM-5GZYH
0,0,ENST00000373020.4,ENSG00000000003.10,13.96,29.20,24.45,14.04,17.03,19.24,25.32,...,26.19,29.63,50.12,23.35,29.76,21.92,13.57,25.24,21.93,24.29
1,1,ENST00000494424.1,ENSG00000000003.10,0.09,0.65,0.58,0.00,0.06,0.19,0.10,...,0.67,0.17,0.80,0.39,0.10,0.00,0.00,0.08,0.22,0.14
2,2,ENST00000496771.1,ENSG00000000003.10,0.39,0.84,1.04,0.31,0.26,0.61,0.24,...,1.70,0.53,0.50,0.81,0.75,0.52,0.23,0.70,0.17,0.73
3,3,ENST00000373031.4,ENSG00000000005.5,0.03,0.04,0.02,0.08,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00
4,4,ENST00000485971.1,ENSG00000000005.5,0.00,0.00,0.00,0.00,0.00,0.00,0.12,...,0.00,0.00,0.00,0.00,0.00,0.00,0.17,0.00,0.00,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196515,196515,ENSTR0000577896.1,ENSGR0000265350.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
196516,196516,ENSTR0000580266.1,ENSGR0000265658.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
196517,196517,ENSTR0000577553.1,ENSGR0000266731.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
196518,196518,ENSTR0000285718.7,ENSGR0000270726.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [15]:
mg = mygene.MyGeneInfo()
geneSyms = mg.querymany(bcds.index.values, scopes='refseq',
                        fields='ensembl.transcript', species='human')



querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

In [65]:
geneSyms2 = mg.querymany(bcds.index.values[1:10], scopes='refseq',
                        fields='ensembl.transcript', species='human')

querying 1-9...done.
Finished.


In [67]:
bcds

,tttttt,tttttc,ttttta,tttttg,tttctt,tttctc,tttcta,tttctg,tttatt,tttatc,...,gggcga,gggcgg,gggagt,gggagc,gggaga,gggagg,gggggt,gggggc,ggggga,gggggg
Protein ID,,,,,,,,,,,,,,,,,,,,,
NP_000005.2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
NP_000006.2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
NP_000007.1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,2,0,0,0,0,0
NP_000008.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NP_000009.1,0,0,0,0,1,0,0,1,0,0,...,0,1,1,0,0,0,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YP_003024034.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YP_003024035.1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
YP_003024036.1,0,0,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [16]:

# subset to specific tissue
cols = ['transcript_id','gene_id'] + sample_dir[tissue]
gtex_data_tissue = gtex_data[cols]
tissue = tissue.replace(" - ", "_").replace(" ", "_").replace("_(BA9)", "")


In [19]:
# reformat
gtex_data_tissue['transcript_id'] = gtex_data_tissue['transcript_id'].str.split('.',expand=True)[0]
gtex_data_tissue_summed = gtex_data_tissue.groupby('transcript_id').sum()



/data/software/anaconda3/envs/bfx-codon/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
# get average per transcript
gtex_avg = gtex_data_tissue_summed.mean(axis=1)




In [21]:
refseqs,weights = get_weights(geneSyms, gtex_avg)

In [74]:
refseqs

['NP_000005.2',
 'NP_000006.2',
 'NP_000007.1',
 'NP_000008.1',
 'NP_000009.1',
 'NP_000010.1',
 'NP_000011.2',
 'NP_000012.1',
 'NP_000013.2',
 'NP_000014.1',
 'NP_000015.1',
 'NP_000016.1',
 'NP_000017.1',
 'NP_000018.2',
 'NP_000019.2',
 'NP_000020.1',
 'NP_000021.1',
 'NP_000022.3',
 'NP_000023.2',
 'NP_000024.2',
 'NP_000026.2',
 'NP_000027.2',
 'NP_000028.3',
 'NP_000029.2',
 'NP_000030.1',
 'NP_000031.1',
 'NP_000032.1',
 'NP_000033.2',
 'NP_000034.1',
 'NP_000035.2',
 'NP_000036.2',
 'NP_000037.2',
 'NP_000038.2',
 'NP_000039.2',
 'NP_000040.1',
 'NP_000041.2',
 'NP_000042.3',
 'NP_000043.4',
 'NP_000044.2',
 'NP_000045.1',
 'NP_000046.1',
 'NP_000047.1',
 'NP_000048.1',
 'NP_000050.2',
 'NP_000052.1',
 'NP_000053.2',
 'NP_000054.2',
 'NP_000054.2',
 'NP_000054.2',
 'NP_000054.2',
 'NP_000054.2',
 'NP_000054.2',
 'NP_000054.2',
 'NP_000055.2',
 'NP_000056.2',
 'NP_000057.3',
 'NP_000058.1',
 'NP_000059.3',
 'NP_000060.2',
 'NP_000061.1',
 'NP_000062.1',
 'NP_000063.2',
 'NP_000

In [22]:
bicod_tab = generate_bicodon_table(refseqs,weights,'bicodon_hifr')

Generated Bicodon Table


In [30]:
bcds_sorted = bcds.loc[refseqs,:]
bcds_weighted = (bcds_sorted.T * weights).sum(axis=1)
bcds_weighted.index = bcds_weighted.index.str.upper()
bcds_weighted.to_frame()

,0
TTTTTT,201294.025600
TTTTTC,197413.184457
TTTTTA,463695.552629
TTTTTG,589960.182457
TTTCTT,204185.213486
...,...
GGGAGG,186424.265029
GGGGGT,417407.055771
GGGGGC,548207.080171
GGGGGA,230671.781486


In [54]:
weights[0:10]

[604.9957142857141,
 29.640742857142868,
 55.65377142857144,
 98.27320000000005,
 1121.9289142857144,
 281.14154285714284,
 2.3808,
 7.3200571428571415,
 2.669771428571428,
 2.528742857142857]

In [38]:
bcds.loc[refseqs,:].T.sum(axis=1)

tttttt    21419
tttttc    21148
ttttta    15819
tttttg    19887
tttctt    24705
          ...  
gggagg    20317
gggggt    17732
gggggc    50763
ggggga    21976
gggggg    12237
Length: 4096, dtype: int64

tttttt    201294.025600
tttttc    197413.184457
ttttta    463695.552629
tttttg    589960.182457
tttctt    204185.213486
              ...      
gggagg    186424.265029
gggggt    417407.055771
gggggc    548207.080171
ggggga    230671.781486
gggggg    134939.226800
Length: 4096, dtype: float64

In [72]:
pd.read_csv('../../references/CoCoPUTs_codon_usage/bicodon_usage/Liver.bicodon.txt',sep='\t').T.sum()

0    1.228309e+08
dtype: float64